In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
import random
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from langchain_ollama.llms import OllamaLLM
# Initialize LLM
llm = OllamaLLM(model="llama3.2")

# Agent class
class LLMAgent(Agent):
    def __init__(self, model):
        super().__init__(model)
        self.llm = llm  # Access the LLM instance

    def make_decision(self, context):
        prompt = f"Given the context: {context}, make a decision based on your goals."
        response = self.llm.invoke(prompt)
        decision = str(response)
        print(f"Agent {self.unique_id} made decision: {decision}")
        return decision

    # def step(self):
    #     context = f"Agent {self.unique_id} is deciding what to do next."
    #     decision = self.make_decision(context)
    #     print(f"Agent {self.unique_id} made decision: {decision}")

# Model class
class LLMModel(Model):
    def __init__(self, width, height, n):
        super().__init__()
        self.num_agents = n
        self.grid = MultiGrid(width, height, True)

        # Create agents
        agents = LLMAgent.create_agents(model=self, n=n)
        x = self.rng.integers(0, self.grid.width, size=(n,))
        y = self.rng.integers(0, self.grid.height, size=(n,))
        for a, i, j in zip(agents, x, y):
            # Add the agent to a random grid cell
            self.grid.place_agent(a, (i, j))

        # Data collector for tracking decision-making
        self.datacollector = DataCollector(
            agent_reporters={"Decision": lambda a: a.make_decision(f"Agent {a.unique_id} is deciding what to do next.")}
        )
        

    def step(self):
        # self.datacollector.collect(self)
        context = """
You are stuck on an island, you have 3 choices:
1) Search for fruits
2) Search for material for shelter
3) Look around the island for the closest ship to contact
Which option would you pick (1,2,3) ?
"""
        self.agents.shuffle_do("make_decision", context)

# Run the model
def run_model():
    model = LLMModel(10, 10, 5)
    for i in range(10):
        print(f"Step {i+1}")
        model.step()
    # agent_data = model.datacollector.get_agent_vars_dataframe()
    # print(agent_data)

if __name__ == "__main__":
    run_model()

Step 1
Agent 4 made decision: Given my goal is to survive and potentially get rescued as soon as possible, I would choose option 3: Look around the island for the closest ship to contact.

Finding a ship with a means of communication could be the quickest way to get help and possibly even rescue. Searching for fruits (option 1) might provide sustenance in the short term but wouldn't address my primary goal of getting off the island. Building shelter (option 2) is also important, but it's not as crucial as finding a means of communication to alert others about my situation.

By prioritizing option 3, I'm taking a proactive step towards potentially getting rescued sooner rather than later.
Agent 3 made decision: A classic survival scenario!

Given my goal is to survive and potentially get rescued, I would choose:

**Option 3: Look around the island for the closest ship to contact**

Why? Well, finding shelter and food are essential, but they don't necessarily guarantee rescue. On the oth

KeyboardInterrupt: 

In [29]:
from mesa import Agent, Model
import random
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

# Initialize LLM
llm = OllamaLLM(model="llama3.2", num_predict=1)

# Agent class with Personality
class LLMAgent(Agent):
    def __init__(self, unique_id, model, personality):
        # Correct initialization of Agent, with the model being passed as the first argument.
        super().__init__(model)  # Pass unique_id first and then model to the Agent base class
        self.llm = llm  # Access the LLM instance
        self.personality = personality  # Personality affects decision-making
        self.stock_held = 3  # Start with no stock holdings
        self.money_held = 70

    def make_decision(self, context):
        context = context.invoke({"amount" :self.stock_held, "personality": self.personality, "money": self.money_held})
        # print(context)
        response = self.llm.invoke(context)
        decision = str(response)
        # print(f"Agent {self.unique_id} made decision: {decision}")
        return decision

    def buy_stock(self, amount, banana_price):
        if self.money_held >= banana_price*amount:
            self.stock_held += amount
            self.money_held -= banana_price*amount
        #     print(f"Agent {self.unique_id} bought {amount} of Banana stock.")
        # else:
        #     print(f"Agent {self.unique_id} could not buy {amount} of Banana stock.")
        

    def sell_stock(self, amount, banana_price):
        if self.stock_held >= amount:
            self.stock_held -= amount
            self.money_held += banana_price*amount
        #     print(f"Agent {self.unique_id} sold {amount} of Banana stock.")
        # else:
        #     print(f"Agent {self.unique_id} tried to sell more than they own.")

    def hold_stock(self):
        # print(f"Agent {self.unique_id} decided to hold their Banana stock.")
        pass

    
    def status(self, banana_price):
        print(f"""Agent {self.unique_id}
Personality: {self.personality}
Stocks: {self.stock_held}
Money: {self.money_held} 
Portfolio value: {self.money_held+self.stock_held*banana_price}
              """)


# Model class with Stock Pricing
class LLMModel(Model):
    def __init__(self, width, height, n, initial_prices=[9.5, 9.8, 10, 10.4, 10.1]):
        super().__init__()
        self.num_agents = n
        self.grid = MultiGrid(width, height, True)
        self.banana_price = initial_prices  # Initial price of the "banana" stock
        self.buy_count = 0  # Track the number of buys
        self.sell_count = 0  # Track the number of sells

        # Create agents with different personalities
        personalities = ["Aggressive", "Cautious", "Risk-Averse", "Optimistic", "Pessimistic"]
        agents = [LLMAgent(i, self, random.choice(personalities)) for i in range(self.num_agents)]
        x = self.rng.integers(0, self.grid.width, size=(n,))
        y = self.rng.integers(0, self.grid.height, size=(n,))
        
        for a, i, j in zip(agents, x, y):
            # Add the agent to a random grid cell
            self.grid.place_agent(a, (i, j))

        # Data collector for tracking decision-making
        self.datacollector = DataCollector(
            agent_reporters={"Decision": lambda a: a.make_decision(f"Agent {a.unique_id} is deciding what to do next.")},
            model_reporters={"BananaPrice": "banana_price"}
        )

    def calculate_stock_price(self):
        # Simple supply-demand model for stock price
        price_change = (self.buy_count - self.sell_count) * 0.1  # Small price fluctuation based on buy/sell activity
        new_price = self.banana_price[-1] + price_change + random.uniform(-3.0, 3.0)
        self.banana_price.append(max(1, new_price))  # Ensure the price doesn't go below 1
        print(f"New Banana stock price: {self.banana_price[-1]}")

    def step(self):
        self.buy_count = 0
        self.sell_count = 0
        
        context = """
The Banana pricing in the past has been: """ + str(self.banana_price[:-1]) + """"
The current price of Banana stock is $""" + str(self.banana_price[-1]) + """"
You have the following options:
1) Buy 1 Banana stock.
2) Buy 3 Banana stock.
3) Buy 5 Banana stock.
4) Sell 1 Banana stock.
5) Sell 3 Banana stock.
6) Sell 5 Banana stock.
7) Hold your position.

Make your decision. Be mindful of your personality and the current banana pricing. Start your response with the number of the option chosen. You currently hold {amount} banana stocks and you have ${money}.
Response no.: 
"""
        
        messages = [
            ("system", "You are a stock trader who is trading banana stock. Your personality type is {personality}"),
            ("human", context)
        ]

        prompt_template = ChatPromptTemplate.from_messages(messages)

        # Agents decide on their actions
        for agent in self.agents:
            decision = agent.make_decision(prompt_template)
            if "1" in decision.lower():
                agent.buy_stock(1, self.banana_price[-1])
                self.buy_count += 1
            elif "2" in decision.lower():
                agent.buy_stock(3, self.banana_price[-1])
                self.buy_count += 3
            elif "3" in decision.lower():
                agent.buy_stock(5, self.banana_price[-1])
                self.buy_count += 5
            elif "4" in decision.lower():
                agent.sell_stock(1, self.banana_price[-1])
                self.sell_count += 1
            elif "5" in decision.lower():
                agent.sell_stock(3, self.banana_price[-1])
                self.sell_count += 3
            elif "6" in decision.lower():
                agent.sell_stock(5, self.banana_price[-1])
                self.sell_count += 5
            else:
                agent.hold_stock()

        # Update the stock price after all actions
        self.calculate_stock_price()
        # self.datacollector.collect(self)

    def status(self):

        for agent in self.agents:
            agent.status(self.banana_price[-1])

# Run the model
def run_model():
    model = LLMModel(10, 10, 20)
    for i in range(50):
        print(f"Step {i+1}")
        model.step()
    model.status()

if __name__ == "__main__":
    run_model()


Step 1
New Banana stock price: 14.200119951081454
Step 2
New Banana stock price: 12.56414184757466
Step 3
New Banana stock price: 11.997866323777167
Step 4
New Banana stock price: 12.22256034927011
Step 5
New Banana stock price: 14.416503974138202
Step 6
New Banana stock price: 15.712069513860191
Step 7
New Banana stock price: 18.628157126187823
Step 8
New Banana stock price: 21.81279800792835
Step 9
New Banana stock price: 19.03577353538308
Step 10
New Banana stock price: 22.407796106356606
Step 11
New Banana stock price: 21.980420697059436
Step 12
New Banana stock price: 24.504624295259458
Step 13
New Banana stock price: 27.469414831834307
Step 14
New Banana stock price: 24.265410008708017
Step 15
New Banana stock price: 24.049817970422012
Step 16
New Banana stock price: 24.80657642133995
Step 17
New Banana stock price: 23.058506514798346
Step 18
New Banana stock price: 21.282473061810496
Step 19
New Banana stock price: 17.442309866292067
Step 20
New Banana stock price: 15.8633755611

In [ ]:
from mesa import Agent, Model
import random
from mesa.space import MultiGrid
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

# Initialize LLM
llm = OllamaLLM(model="llama3.2", num_predict=1)

# Agent class with Personality
class LLMAgent(Agent):
    def __init__(self, unique_id, model, personality):
        # Correct initialization of Agent, with the model being passed as the first argument.
        super().__init__(model)  # Pass unique_id first and then model to the Agent base class
        self.llm = llm  # Access the LLM instance
        self.personality = personality  # Personality affects decision-making
        self.stock_held = 3  # Start with no stock holdings
        self.money_held = 70

    def make_decision(self, context):
        context = context.invoke({"amount" :self.stock_held, "personality": self.personality, "money": self.money_held})
        response = self.llm.invoke(context)
        decision = str(response)
        return decision

    def buy_stock(self, amount, banana_price):
        if self.money_held >= banana_price*amount:
            self.stock_held += amount
            self.money_held -= banana_price*amount
            print(f"Agent {self.unique_id} bought {amount} of Banana stock.")
        else:
            print(f"Agent {self.unique_id} could not buy {amount} of Banana stock.")
        

    def sell_stock(self, amount, banana_price):
        if self.stock_held >= amount:
            self.stock_held -= amount
            self.money_held += banana_price*amount
            print(f"Agent {self.unique_id} sold {amount} of Banana stock.")
        else:
            print(f"Agent {self.unique_id} tried to sell more than they own.")

    def hold_stock(self):
        print(f"Agent {self.unique_id} decided to hold their Banana stock.")

    
    def status(self, banana_price):
        print(f"""Agent {self.unique_id}
Personality: {self.personality}
Stocks: {self.stock_held}
Money: {self.money_held} 
Portfolio value: {self.money_held+self.stock_held*banana_price}
              """)


# Model class with Stock Pricing
class LLMModel(Model):
    def __init__(self, width, height, n, initial_prices=[9.5, 9.8, 10, 10.4, 10.1]):
        super().__init__()
        self.num_agents = n
        self.grid = MultiGrid(width, height, True)
        self.banana_price = initial_prices  # Initial price of the "banana" stock
        self.buy_count = 0  # Track the number of buys
        self.sell_count = 0  # Track the number of sells

        # Create agents with different personalities
        personalities = ["Aggressive", "Cautious", "Risk-Averse", "Optimistic", "Pessimistic"]
        agents = [LLMAgent(i, self, random.choice(personalities)) for i in range(self.num_agents)]
        x = self.rng.integers(0, self.grid.width, size=(n,))
        y = self.rng.integers(0, self.grid.height, size=(n,))
        
        for a, i, j in zip(agents, x, y):
            # Add the agent to a random grid cell
            self.grid.place_agent(a, (i, j))

    def calculate_stock_price(self):
        # Simple supply-demand model for stock price
        price_change = (self.buy_count - self.sell_count) * 0.1  # Small price fluctuation based on buy/sell activity
        new_price = self.banana_price[-1] + price_change + random.uniform(-3.0, 3.0)
        self.banana_price.append(max(1, new_price))  # Ensure the price doesn't go below 1
        print(f"New Banana stock price: {self.banana_price[-1]}")

    def step(self):
        self.buy_count = 0
        self.sell_count = 0
        
        context = """
The Banana pricing in the past has been: """ + str(self.banana_price[:-1]) + """"
The current price of Banana stock is $""" + str(self.banana_price[-1]) + """"
You have the following options:
1) Buy 1 Banana stock.
2) Buy 3 Banana stock.
3) Buy 5 Banana stock.
4) Sell 1 Banana stock.
5) Sell 3 Banana stock.
6) Sell 5 Banana stock.
7) Hold your position.

Make your decision. Be mindful of your personality and the current banana pricing. Start your response with the number of the option chosen. You currently hold {amount} banana stocks and you have ${money}.
Response no.: 
"""
        
        messages = [
            ("system", "You are a stock trader who is trading banana stock. Your personality type is {personality}"),
            ("human", context)
        ]

        prompt_template = ChatPromptTemplate.from_messages(messages)

        # Agents decide on their actions
        for agent in self.agents:
            decision = agent.make_decision(prompt_template)
            if "1" in decision.lower():
                agent.buy_stock(1, self.banana_price[-1])
                self.buy_count += 1
            elif "2" in decision.lower():
                agent.buy_stock(3, self.banana_price[-1])
                self.buy_count += 3
            elif "3" in decision.lower():
                agent.buy_stock(5, self.banana_price[-1])
                self.buy_count += 5
            elif "4" in decision.lower():
                agent.sell_stock(1, self.banana_price[-1])
                self.sell_count += 1
            elif "5" in decision.lower():
                agent.sell_stock(3, self.banana_price[-1])
                self.sell_count += 3
            elif "6" in decision.lower():
                agent.sell_stock(5, self.banana_price[-1])
                self.sell_count += 5
            else:
                agent.hold_stock()

        # Update the stock price after all actions
        self.calculate_stock_price()

    def status(self):

        for agent in self.agents:
            agent.status(self.banana_price[-1])

# Run the model
def run_model():
    model = LLMModel(10, 10, 20)
    for i in range(40):
        print(f"Step {i+1}")
        model.step()
    model.status()

if __name__ == "__main__":
    run_model()


Step 1
New Banana stock price: 14.015498366760788
Step 2
New Banana stock price: 14.054284774702072
Step 3
New Banana stock price: 16.274461440462634
Step 4
New Banana stock price: 19.313909785507548
Step 5
New Banana stock price: 20.48800136873774
Step 6
New Banana stock price: 22.094819789446888
Step 7
New Banana stock price: 25.082399311186272
Step 8
New Banana stock price: 24.634612765317186
Step 9
New Banana stock price: 23.995354123688074
Step 10
New Banana stock price: 21.526374835289005
Step 11
New Banana stock price: 21.585746450835458
Step 12
New Banana stock price: 22.614136748185643
Step 13
New Banana stock price: 25.375845028976592
Step 14
New Banana stock price: 25.435604999121445
Step 15
New Banana stock price: 24.28051225438829
Step 16
New Banana stock price: 24.64879903619729
Step 17
New Banana stock price: 25.268269145132777
Step 18
New Banana stock price: 25.157103559563325
Step 19
New Banana stock price: 24.13405630738601
Step 20
New Banana stock price: 26.293081556

In [ ]:
from mesa import Agent, Model
import random
from mesa.space import MultiGrid
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

# Initialize LLM
llm = OllamaLLM(model="llama3.2", num_predict=1)

# Agent class with Personality
class LLMAgent(Agent):
    def __init__(self, unique_id, model, personality):
        # Correct initialization of Agent, with the model being passed as the first argument.
        super().__init__(model)  # Pass unique_id first and then model to the Agent base class
        self.llm = llm  # Access the LLM instance
        self.personality = personality  # Personality affects decision-making
        self.stock_held = 3  # Start with no stock holdings
        self.money_held = 70

    def make_decision(self, context):
        context = context.invoke({"amount" :self.stock_held, "personality": self.personality, "money": self.money_held})
        response = self.llm.invoke(context)
        decision = str(response)
        return decision

    def buy_stock(self, amount, banana_price):
        if self.money_held >= banana_price*amount:
            self.stock_held += amount
            self.money_held -= banana_price*amount
            print(f"Agent {self.unique_id} bought {amount} of Banana stock.")
        else:
            print(f"Agent {self.unique_id} could not buy {amount} of Banana stock.")
        

    def sell_stock(self, amount, banana_price):
        if self.stock_held >= amount:
            self.stock_held -= amount
            self.money_held += banana_price*amount
            print(f"Agent {self.unique_id} sold {amount} of Banana stock.")
        else:
            print(f"Agent {self.unique_id} tried to sell more than they own.")

    def hold_stock(self):
        print(f"Agent {self.unique_id} decided to hold their Banana stock.")

    
    def status(self, banana_price):
        print(f"""Agent {self.unique_id}
Personality: {self.personality}
Stocks: {self.stock_held}
Money: {self.money_held} 
Portfolio value: {self.money_held+self.stock_held*banana_price}
              """)


# Model class with Stock Pricing
class LLMModel(Model):
    def __init__(self, width, height, n, initial_prices=[9.5, 9.8, 10, 10.4, 10.1]):
        super().__init__()
        self.num_agents = n
        self.grid = MultiGrid(width, height, True)
        self.banana_price = initial_prices  # Initial price of the "banana" stock
        self.buy_count = 0  # Track the number of buys
        self.sell_count = 0  # Track the number of sells

        # Create agents with different personalities
        personalities = ["Aggressive", "Cautious", "Risk-Averse", "Optimistic", "Pessimistic"]
        agents = [LLMAgent(i, self, random.choice(personalities)) for i in range(self.num_agents)]
        x = self.rng.integers(0, self.grid.width, size=(n,))
        y = self.rng.integers(0, self.grid.height, size=(n,))
        
        for a, i, j in zip(agents, x, y):
            # Add the agent to a random grid cell
            self.grid.place_agent(a, (i, j))

    def calculate_stock_price(self):
        # Simple supply-demand model for stock price
        price_change = (self.buy_count - self.sell_count) * 0.05  # Small price fluctuation based on buy/sell activity
        new_price = self.banana_price[-1] + price_change + random.uniform(-2.0, 2.0)
        self.banana_price.append(max(1, new_price))  # Ensure the price doesn't go below 1
        print(f"New Banana stock price: {self.banana_price[-1]}")

    def step(self):
        self.buy_count = 0
        self.sell_count = 0
        
        context = """
The Banana pricing in the past has been: """ + str(self.banana_price[:-1]) + """"
The current price of Banana stock is $""" + str(self.banana_price[-1]) + """"
You have the following options:
1) Buy 1 Banana stock.
2) Buy 3 Banana stock.
3) Buy 5 Banana stock.
4) Sell 1 Banana stock.
5) Sell 3 Banana stock.
6) Sell 5 Banana stock.
7) Hold your position.

Make your decision. Be mindful of your personality and the current banana pricing. Start your response with the number of the option chosen. You currently hold {amount} banana stocks and you have ${money}.
Response no.: 
"""
        
        messages = [
            ("system", "You are a stock trader who is trading banana stock. Your personality type is {personality}"),
            ("human", context)
        ]

        prompt_template = ChatPromptTemplate.from_messages(messages)

        # Agents decide on their actions
        for agent in self.agents:
            decision = agent.make_decision(prompt_template)
            if "1" in decision.lower():
                agent.buy_stock(1, self.banana_price[-1])
                self.buy_count += 1
            elif "2" in decision.lower():
                agent.buy_stock(3, self.banana_price[-1])
                self.buy_count += 3
            elif "3" in decision.lower():
                agent.buy_stock(5, self.banana_price[-1])
                self.buy_count += 5
            elif "4" in decision.lower():
                agent.sell_stock(1, self.banana_price[-1])
                self.sell_count += 1
            elif "5" in decision.lower():
                agent.sell_stock(3, self.banana_price[-1])
                self.sell_count += 3
            elif "6" in decision.lower():
                agent.sell_stock(5, self.banana_price[-1])
                self.sell_count += 5
            else:
                agent.hold_stock()

        # Update the stock price after all actions
        self.calculate_stock_price()

    def status(self):

        for agent in self.agents:
            agent.status(self.banana_price[-1])

# Run the model
def run_model():
    model = LLMModel(10, 10, 20)
    for i in range(15):
        print(f"Step {i+1}")
        model.step()
    model.status()

if __name__ == "__main__":
    run_model()


Step 1
Agent 1 decided to hold their Banana stock.
Agent 2 bought 1 of Banana stock.
Agent 3 decided to hold their Banana stock.
Agent 4 sold 1 of Banana stock.
Agent 5 sold 1 of Banana stock.
Agent 6 bought 1 of Banana stock.
Agent 7 bought 3 of Banana stock.
Agent 8 bought 1 of Banana stock.
Agent 9 sold 1 of Banana stock.
Agent 10 sold 1 of Banana stock.
Agent 11 bought 1 of Banana stock.
Agent 12 sold 1 of Banana stock.
Agent 13 bought 3 of Banana stock.
Agent 14 sold 1 of Banana stock.
Agent 15 bought 1 of Banana stock.
Agent 16 bought 1 of Banana stock.
Agent 17 bought 1 of Banana stock.
Agent 18 bought 1 of Banana stock.
Agent 19 bought 1 of Banana stock.
Agent 20 bought 1 of Banana stock.
New Banana stock price: 10.870564029960814
Step 2
Agent 1 sold 1 of Banana stock.
Agent 2 bought 1 of Banana stock.
Agent 3 sold 1 of Banana stock.
Agent 4 decided to hold their Banana stock.
Agent 5 sold 1 of Banana stock.
Agent 6 bought 5 of Banana stock.
Agent 7 decided to hold their Banana